In [1]:
# Notes and play code.

In [34]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

query = 'I love tennis'
text = 'andy murry is my favourite tennis player'
max_length = 512

In [35]:
x = tokenizer.encode_plus(text=query,
                          text_pair=text,
                          max_length=max_length,
                          add_special_tokens=True,
                          pad_to_max_length=True)

In [36]:
tokenizer.convert_ids_to_tokens(x['input_ids'])

['<s>',
 'ĠI',
 'Ġlove',
 'Ġtennis',
 '</s>',
 '</s>',
 'Ġand',
 'y',
 'Ġmur',
 'ry',
 'Ġis',
 'Ġmy',
 'Ġfavourite',
 'Ġtennis',
 'Ġplayer',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',


In [21]:
x['input_ids']

[0,
 38,
 657,
 5919,
 2,
 2,
 8,
 219,
 22802,
 1506,
 16,
 127,
 5548,
 5919,
 869,
 2,
 1,
 1]

In [56]:
from transformers import RobertaModel, RobertaConfig

bert_1  = RobertaModel.from_pretrained('roberta-base')

In [57]:
bert_1.state_dict()

199

In [58]:
bert_2  = bert_1.state_dict()

In [59]:
bert_1  = RobertaModel.from_pretrained('roberta-base')

AttributeError: 'collections.OrderedDict' object has no attribute 'state_dict'

In [65]:
from transformers import RobertaConfig, RobertaModel
from transformers import BertModel, BertPreTrainedModel
from torch import nn, sigmoid
from torch.nn import MSELoss

class RoBERTaMultiTaskRanker(BertPreTrainedModel):
    """ Bert Multi-Task ranking model for passage and entity ranking. """

    valid_head_flags = ['entity', 'passage']

    def __init__(self, config):
        super().__init__(config)
        # Initialise BERT setup.
        self.bert = RobertaModel(config)
        # Dropout standard of 0.1.
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # Head for passage ranking between 0 (not relevant) & 1 (relevant)
        self.passage_head = nn.Linear(config.hidden_size, 1)
        # Head for entity ranking between 0 (not relevant) & 1 (relevant)
        self.entity_head = nn.Linear(config.hidden_size, 1)
        # Initialise BERT weights.
        #TODO - try without?
        self.init_weights()


    def __get_BERT_outputs(self, input_ids, attention_mask):
        """ Returns BERT outputs (last_hidden_state, pooler_output, hidden_states, attentions) """
        return self.bert(input_ids=input_ids, attention_mask=attention_mask)


    def __get_BERT_cls_vector(self, input_ids, attention_mask):
        """ Returns BERT pooled_output (i.e. CLS vector) applying dropout. """
        # Get BERT outputs.
        outputs = self.__get_BERT_outputs(input_ids=input_ids, attention_mask=attention_mask)
        # Apply dropout to pooled_output (i.e. CLS vector) and apply dropout.
        pooled_output = outputs[1]
        return self.dropout(pooled_output)


    def __get_MSE(self, logits, labels):
        """ Calculate mean squared error (MSE) from logits given labels. """
        loss_fct = MSELoss()
        return loss_fct(logits.view(-1), labels.view(-1))


    def forward_head(self, head_flag='passage', input_ids=None, attention_mask=None, labels=None):
        """ Forward pass over BERT + passage head. Returns loss and logits. """
        # Get BERT CLS vector.
        cls_vector = self.__get_BERT_cls_vector(input_ids=input_ids, attention_mask=attention_mask)

        assert head_flag in self.valid_head_flags, "head_flag: {}, valid_head_flags: {}".format(head_flag, self.valid_head_flags)
        # Calculate logits.
        if head_flag == 'passage':
            logits = sigmoid(self.passage_head(cls_vector))
        elif head_flag == 'entity':
            logits = sigmoid(self.entity_head(cls_vector))
        else:
            "NOT VALID HEAD SELECTION"
            raise

        # Calculate loss.
        loss = self.__get_MSE(logits=logits, labels=labels)

        return loss, logits

In [66]:
model = RoBERTaMultiTaskRanker.from_pretrained('roberta-base')

In [67]:
model.state_dict()

OrderedDict([('bert.embeddings.word_embeddings.weight',
              tensor([[-0.0126,  0.0095,  0.0270,  ..., -0.0012,  0.0138,  0.0178],
                      [-0.0008,  0.0244,  0.0285,  ...,  0.0297,  0.0166,  0.0146],
                      [-0.0200, -0.0463,  0.0230,  ...,  0.0167, -0.0118, -0.0332],
                      ...,
                      [-0.0067,  0.0099, -0.0061,  ..., -0.0072,  0.0035,  0.0310],
                      [-0.0071,  0.0224,  0.0025,  ...,  0.0501,  0.0070, -0.0170],
                      [-0.0079,  0.0208, -0.0538,  ..., -0.0250, -0.0117,  0.0103]])),
             ('bert.embeddings.position_embeddings.weight',
              tensor([[ 0.0163, -0.0116,  0.0265,  ...,  0.0026, -0.0273,  0.0010],
                      [ 0.0234,  0.0013,  0.0010,  ..., -0.0086,  0.0282,  0.0274],
                      [-0.0043, -0.0170, -0.0199,  ..., -0.0054,  0.0208, -0.0211],
                      ...,
                      [-0.0213,  0.0042,  0.0075,  ...,  0.0259,  0.022

In [68]:
model_dir = '/Users/iain/LocalStorage/coding/github/multi-task-ranking/data/temp/model_write'
model.save_pretrained(model_dir)

In [69]:
model_read = RoBERTaMultiTaskRanker.from_pretrained(model_dir)

In [70]:
model_read.state_dict()

OrderedDict([('bert.embeddings.word_embeddings.weight',
              tensor([[-0.0126,  0.0095,  0.0270,  ..., -0.0012,  0.0138,  0.0178],
                      [-0.0008,  0.0244,  0.0285,  ...,  0.0297,  0.0166,  0.0146],
                      [-0.0200, -0.0463,  0.0230,  ...,  0.0167, -0.0118, -0.0332],
                      ...,
                      [-0.0067,  0.0099, -0.0061,  ..., -0.0072,  0.0035,  0.0310],
                      [-0.0071,  0.0224,  0.0025,  ...,  0.0501,  0.0070, -0.0170],
                      [-0.0079,  0.0208, -0.0538,  ..., -0.0250, -0.0117,  0.0103]])),
             ('bert.embeddings.position_embeddings.weight',
              tensor([[ 0.0163, -0.0116,  0.0265,  ...,  0.0026, -0.0273,  0.0010],
                      [ 0.0234,  0.0013,  0.0010,  ..., -0.0086,  0.0282,  0.0274],
                      [-0.0043, -0.0170, -0.0199,  ..., -0.0054,  0.0208, -0.0211],
                      ...,
                      [-0.0213,  0.0042,  0.0075,  ...,  0.0259,  0.022